# Missing Data

Author: Luke Moraglia

This notebook gives an overview of methods to handle missing data. It covers:
1. Dropping rows or columns that contain NA values
2. Mean and median imputation
3. Multivariate feature imputation with `IterativeImputer`
4. Nearest neighbors imputation with `KNNImputer`

## Types of missing data

*The information in this section is summarized from the Bhaskaran and Smeeth article linked below.*    
Missing data can be categorized in three ways: missing completely at random (MCAR), missing at random (MAR), and missing not at random (MNAR).
When a variable has observations that are MCAR, the missing observations are a random subset of the observations. This means that missing observations and present observations have similar distributions and there are not systematic differences between the missing and present observations. For example, if a sensor collecting data is randomly faulty, we would expect our missing observations to be a truly random subset of all the observations. 
When a variable has observations that are MAR, there could be systematic differences between the missing and present observations leading to different distributions, but these differences can be explained by other observed variables. For instance, we might have blood pressure data that are missing more often for younger patients, because a doctor is more likely to be certain to collect BP data on his older patients. Our younger patients are also likely to have lower blood pressures than the older patients, so we are guessing that our missing data will have a different distribution (lower mean) than our present data. This difference in distributions might make it seem like the data are not missing at random, but since we have also recorded the age of the patients, we can use age as a variable that explains the difference in blood pressure. This makes blood pressure MAR, which is valid for missing data methods like multiple imputation.  
For variables with observations MNAR, imputation will be invalid since there are systematic differences between the missing and present observations that cannot be explained by other variables. For instance, a doctor is more likely to record BMI for a patient who is noticibly overweight, meaning that missing data will tend to have a distribution with lower BMIs. This cannot be explained by any other variables such as age or sex, so these data are MNAR. 

(Re)Sources:
- [Abhishek Thakur and Rob Mulla missing data livestream](https://www.youtube.com/watch?v=EYySNJU8qR0&t=3127s) on which much of this code is based
- [Jason Brownlee's Machine Learning Mastery](https://machinelearningmastery.com/handle-missing-data-python/) where the dataset inspiration comes from
- [sklearn](https://scikit-learn.org/stable/modules/impute.html#)
- [Article by Bhaskaran & Smeeth](https://academic.oup.com/ije/article/43/4/1336/2938944?login=true) on the difference between MCAR, MAR, and MNAR

# Library Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

from sklearn.datasets import fetch_covtype
from numpy.random import default_rng
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

# Load Data / Investigate Missing Data
We will use the Diabetes dataset available from Jason Brownlee (link above). We have 8 numeric variables as our input variables and a binary outcome of diabetes present or absent.

In [2]:
columns = ["pregnant", "glucose", "BP", "skin_thick", "insulin", "BMI", "diab_ped_func", "age", "diabetes"]

In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv", names=columns)

In [4]:
data.describe()

,pregnant,glucose,BP,skin_thick,insulin,BMI,diab_ped_func,age,diabetes
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In this data, missing values on several columns have been coded with '0'. This is confusing because is a numeric value and Python cannot treat it as missing. We can switch these values to `NaN` so that Python/pandas can treat it as a missing value. 

In [5]:
cols_missing = ["glucose", "BP", "skin_thick", "insulin", "BMI"]

In [6]:
# replace zeroes in these columns with NaN
data[cols_missing] = data[cols_missing].replace(0, np.nan)
data.head()

,pregnant,glucose,BP,skin_thick,insulin,BMI,diab_ped_func,age,diabetes
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


We can check how many missing values there are in each variable, as well as the percentage of missing values in each.

In [7]:
data.isna().sum()

pregnant           0
glucose            5
BP                35
skin_thick       227
insulin          374
BMI               11
diab_ped_func      0
age                0
diabetes           0
dtype: int64

In [8]:
data.isna().mean()

pregnant         0.000000
glucose          0.006510
BP               0.045573
skin_thick       0.295573
insulin          0.486979
BMI              0.014323
diab_ped_func    0.000000
age              0.000000
diabetes         0.000000
dtype: float64

Finally we'll do a train test split.

In [9]:
X = data.iloc[:,:8]
y = data.iloc[:,8]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Use missingness as a predictor

We can create a new variable from each variable that has missing values. These new variables are indicator variables that are "True" if the observation had a missing value for that variable. This can give us an idea of if the fact that a value is missing is informative for predicting diabetes.

In [11]:
X_missing_tag = X_train[cols_missing].isna()

In [12]:
X_missing_tag.columns = [f"{c}_missing" for c in cols_missing]

In [13]:
X_missing_tag.head()

,glucose_missing,BP_missing,skin_thick_missing,insulin_missing,BMI_missing
353,False,False,False,False,False
711,False,False,False,False,False
373,False,False,False,False,False
46,False,False,True,True,False
682,False,False,False,False,False


In [14]:
def model_eval(model, X, y):
    y_pred = model.predict_proba(X)[:, 1]
    print(f"ROC-AUC: {roc_auc_score(y, y_pred)}")

In [15]:
lr = LogisticRegressionCV(scoring="accuracy")

lr.fit(X_missing_tag, y_train)
lr.score(X_missing_tag, y_train)

model_eval(lr, X_missing_tag, y_train)

ROC-AUC: 0.5232827102803738


A quick logistic regression shows a ROC-AUC that is about what we would expect by chance alone, so the presence or absence of missing data does not appear to be predictive in this case.

# Ways to handle missing data
We will investigate a few ways to handle missing data. For each method, we'll see how a Gradient Boosting Classifier performs in terms of ROC-AUC on the data. In the video linked above, Rob Mulla is asked a few questions about what method should be used for a specific situation, and his answer consistently turned to the practical solution of "Try them all and pick the one that performs best in your cross-validation." 

In [16]:
model = GradientBoostingClassifier(random_state=42,
                                   max_depth = 2,
                                   n_estimators=20
                                   )


## Drop rows or columns with missing data
One way to get data with no missing values is to drop any rows or any columns that have missing data. This has some clear disadvantages, the main one being that you are throwing away potentially valuable data. It also could be impractical in a real world setting where new observations will also have missing values. It would probably make more sense to have a model that is able to handle missing data rather than getting rid of them.

In [17]:
# Drop NA rows
data_drop_rows = data.dropna(axis=0)
X_drop_rows = data_drop_rows.iloc[:,:8]
y_drop_rows = data_drop_rows.iloc[:,8]
X_drop_rows.shape

(392, 8)

In [18]:
model.fit(X_drop_rows, y_drop_rows)
model_eval(model, X_drop_rows, y_drop_rows)

ROC-AUC: 0.9049324721080447


In [19]:
# Drop NA columns
data_drop_cols = data.dropna(axis=1)
X_drop_cols = data_drop_cols.drop("diabetes", axis=1)
y_drop_cols = data_drop_cols["diabetes"]
X_drop_cols.shape

(768, 3)

In [20]:
model.fit(X_drop_cols, y_drop_cols)
model_eval(model, X_drop_cols, y_drop_cols)

ROC-AUC: 0.758794776119403


## Mean / median imputation

We can replace (impute) missing values with some other value. Two common values for numeric predictors are the mean and median of the variable. If we had categorical predictors we could also use mode imputation, though this feels less intuitive than the mean and median situation.

The `SimpleImputer` is a class to easily impute the mean, median, or mode for each variable in the data.

In [21]:
# Impute with mean
imptr = SimpleImputer(strategy="mean", add_indicator=False)

In [22]:
X_train_mean_imp = imptr.fit_transform(X_train)
X_test_mean_imp = imptr.transform(X_test)

In [23]:
model.fit(X_train_mean_imp, y_train)
model_eval(model, X_test_mean_imp, y_test)

ROC-AUC: 0.8037037037037036


In [24]:
# Impute with median
imptr = SimpleImputer(strategy="median", add_indicator=False)

In [25]:
X_train_med_imp = imptr.fit_transform(X_train)
X_test_med_imp = imptr.transform(X_test)

In [26]:
model.fit(X_train_med_imp, y_train)
model_eval(model, X_test_med_imp, y_test)

ROC-AUC: 0.8037037037037036


## Multivariate imputation
Mean and median imputation rely only on the values of the variable being imputed. In contrast, we could use the values of the other input variables to try to predict the variable that is being imputed. A regressor is fit that predicts that variable being imputed from the other variables, and the estimates for the missing values from this regressor are used as the imputed values.

The `IterativeImputer` from scikit-learn uses a Bayesian Ridge Regressor by default, but you can be used with any estimator.

In [27]:
# Iterative imputer
it_imputer = IterativeImputer(max_iter=10, random_state=42)
X_train_it_imp = it_imputer.fit_transform(X_train)
X_test_it_imp = it_imputer.transform(X_test)

In [28]:
model.fit(X_train_it_imp, y_train)
model_eval(model, X_test_it_imp, y_test)

ROC-AUC: 0.7957407407407406


## Nearest Neighbor Imputation
We could also impute values by using the average of the observation's $k$ nearest neighbors. This is implemented by the `KNNImputer` with the default of `n_neighbors=5`.

In [29]:
# KNN imputer
knn_imputer = KNNImputer()
X_train_knn_imp = knn_imputer.fit_transform(X_train)
X_test_knn_imp = knn_imputer.transform(X_test)

In [30]:
model.fit(X_train_knn_imp, y_train)
model_eval(model, X_test_knn_imp, y_test)

ROC-AUC: 0.8158333333333333
